In [403]:
!pip install google-colab-selenium

In [404]:
import google_colab_selenium as gs

driver = gs.Chrome()

<IPython.core.display.Javascript object>

# **GET STANDINGS FROM ESPN**

In [405]:
import time

url = "https://fantasy.espn.com/basketball/league/standings?leagueId=121416459"
driver.get(url)

# Wait for JavaScript to load content
time.sleep(7)

html_content = driver.page_source
driver.quit()

In [406]:
from bs4 import BeautifulSoup

NUM_TEAMS = 11

soup = BeautifulSoup(html_content, 'html.parser')

In [407]:
def get_team_names(soup):
  team_names_spans = soup.find_all(class_='teamName')
  return [element.text.strip() for element in team_names_spans[:NUM_TEAMS]]

In [408]:
def get_category_headers(soup):
  categories_th = soup.find_all(class_='Table__TH')
  return [element.text for element in categories_th[-13:-1] if element.text != "" and element.text != "All"]

In [409]:
def get_rows_values(soup):
  table_tr = soup.find_all(class_='Table__TR')
  ROW_START = -24
  ROW_END = ROW_START + NUM_TEAMS
  rows_values = []
  for table_row in table_tr[ROW_START:ROW_END]:
    tds = table_row.findAll()
    rows_values.append([element.text for element in tds[::2]])
  return rows_values

In [410]:
def get_gps(soup):
  table_tr = soup.find_all(class_='Table__TR')
  return [table_row.findAll()[0].text for table_row in table_tr[-NUM_TEAMS:]]

In [411]:
team_names = get_team_names(soup)
rows_values = get_rows_values(soup)
gps = get_gps(soup)
full_rows = [[team_name] + row_values + [gp] for team_name, row_values, gp in zip(team_names, rows_values, gps)]

categories = get_category_headers(soup)


# **CURRENT STANDINGS (TOTALS)**



In [426]:
import pandas as pd

pd.set_option('display.multi_sparse', False)

In [413]:
df = pd.DataFrame(full_rows, columns=categories)
df[df.columns[1:]] = df[df.columns[1:]].apply(pd.to_numeric, errors='coerce')
df

,Team,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,GP
0,TRUST THE PROCESS,0.4865,1.0000,9,12,9,1,2,52,3
1,Gil's Great Team,0.4468,0.6250,5,29,9,1,2,52,3
2,Roy's Rowdy Team,0.7778,0.5000,8,4,10,1,1,37,1
3,Dor's Team,0.4000,0.8824,1,21,7,2,3,40,2
4,Oriel's Otherworldly Team,0.5000,1.0000,6,8,3,1,0,45,2
5,Alon's Astounding Team,0.7778,0.0000,4,4,4,1,0,18,1
6,AviorYNWA,0.3043,0.5000,3,4,8,4,0,19,2
7,Ofer Yannay Money,0.5000,0.7143,1,9,4,0,0,16,1
8,Asaf's Astounding Team,0.5556,1.0000,1,7,3,0,0,12,1
9,Amihai's Angry Team,0.6667,0.6667,0,4,3,0,0,12,1


# **PER GAME AVERAGES**

In [414]:
for col in df.columns[3:-1]:
    df[col] /= df['GP']
df = df.round(4)
df

,Team,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,GP
0,TRUST THE PROCESS,0.4865,1.0000,3.0000,4.0000,3.0,0.3333,0.6667,17.3333,3
1,Gil's Great Team,0.4468,0.6250,1.6667,9.6667,3.0,0.3333,0.6667,17.3333,3
2,Roy's Rowdy Team,0.7778,0.5000,8.0000,4.0000,10.0,1.0000,1.0000,37.0000,1
3,Dor's Team,0.4000,0.8824,0.5000,10.5000,3.5,1.0000,1.5000,20.0000,2
4,Oriel's Otherworldly Team,0.5000,1.0000,3.0000,4.0000,1.5,0.5000,0.0000,22.5000,2
5,Alon's Astounding Team,0.7778,0.0000,4.0000,4.0000,4.0,1.0000,0.0000,18.0000,1
6,AviorYNWA,0.3043,0.5000,1.5000,2.0000,4.0,2.0000,0.0000,9.5000,2
7,Ofer Yannay Money,0.5000,0.7143,1.0000,9.0000,4.0,0.0000,0.0000,16.0000,1
8,Asaf's Astounding Team,0.5556,1.0000,1.0000,7.0000,3.0,0.0000,0.0000,12.0000,1
9,Amihai's Angry Team,0.6667,0.6667,0.0000,4.0000,3.0,0.0000,0.0000,12.0000,1


# **RANKINGS (BASED ON AVERAGES)**


In [450]:
exclude_columns = ['Team', 'GP', 'MOVES']
columns_for_ranking = [col for col in df.columns[1:] if col not in exclude_columns]

ranked = df.copy()


ranked[columns_for_ranking] = df[columns_for_ranking].rank()
ranked['Total_Points'] = ranked.iloc[:, 1:-1].sum(axis=1)
ranked.index = ranked.index + 1

ranked

,Team,FG%,FT%,3PM,REB,AST,STL,BLK,PTS,GP,Total_Points
1,TRUST THE PROCESS,4.0,10.0,8.5,5.0,4.5,5.5,8.5,6.5,3,52.5
2,Gil's Great Team,3.0,5.0,6.0,10.0,4.5,5.5,8.5,6.5,3,49.0
3,Roy's Rowdy Team,10.5,3.5,11.0,5.0,11.0,9.0,10.0,11.0,1,71.0
4,Dor's Team,2.0,8.0,2.0,11.0,7.0,9.0,11.0,9.0,2,59.0
5,Oriel's Otherworldly Team,5.5,10.0,8.5,5.0,1.0,7.0,4.0,10.0,2,51.0
6,Alon's Astounding Team,10.5,1.5,10.0,5.0,9.0,9.0,4.0,8.0,1,57.0
7,AviorYNWA,1.0,3.5,5.0,2.0,9.0,11.0,4.0,1.0,2,36.5
8,Ofer Yannay Money,5.5,7.0,3.5,9.0,9.0,2.5,4.0,4.5,1,45.0
9,Asaf's Astounding Team,8.0,10.0,3.5,8.0,4.5,2.5,4.0,2.5,1,43.0
10,Amihai's Angry Team,9.0,6.0,1.0,5.0,4.5,2.5,4.0,2.5,1,34.5
